# Boosting

###High level summary
Train a series of weak decision tree learners, with each successive tree attacking the residual from all previous trees.

##Algorithm
Train a weak decision tree $F^0$ to fit the data
$$\renewcommand{\vec}[1]{\mathbf{#1}}
F^0(\vec{x_1}) \qquad y_1 \\
F^0(\vec{x_2}) \qquad y_2 \\
\vdots \qquad \vdots \\
F^0(\vec{x_n}) \qquad y_n$$

This will lead to residuals
$$r^0(\vec{x_1}) = y_1 - F^0(\vec{x_1})\\ r^0(\vec{x_2}) = y_2 - F^0(\vec{x_2})\\ \vdots \\r^0(\vec{x_n}) = y_n - F^0(\vec{x_n})$$

Now, fit a second tree $F^1$ to the residual so that the new overall prediction $F(X) = F^0(X) + F^1(X)$

$$
F^1(\vec{x_1}) \qquad y_1 - F^0(\vec{x_1}) \\
F^1(\vec{x_2}) \qquad y_2 - F^0(\vec{x_2}) \\
\vdots \qquad \vdots \\
F^1(\vec{x_n}) \qquad y_n - F^0(\vec{x_n})$$

This will lead to residuals
$$r^1(\vec{x_1}) = y_1 - \left(F^0(\vec{x_1}) + F^1(\vec{x_1})\right)\\ r^1(\vec{x_2}) = y_2 - \left(F^0(\vec{x_2}) + F^1(\vec{x_2})\right)\\ \vdots \\r^1(\vec{x_n}) = y_n - \left(F^0(\vec{x_n}) + F^1(\vec{x_n})\right)$$

Keep fitting decision trees until some criterion is met or max trees have been trained.

**Ok, but why is this called gradient boosting?**

Consider the loss function
$$L(y, F(\vec{x})) = \frac{(y-F(\vec{x}))^2}{2}$$

Which leads to the cost function
$$J = \sum_i L(y_i, F(\vec{x_i}))$$

The negative gradient of this cost function is
$$-\frac{\partial J}{\partial{F(\vec{x_i})}} = y_i - F(\vec{x_i})$$

which is exactly the residual used in the boosting algorithm described above. The residuals are really just the negative gradients of the cost function!

### Other loss functions

#### Absolute loss


$$L(y, F) = \left|y-F\right| \\ -g(\vec{x_i}) = -\frac{\partial J}{\partial{F(\vec{x_i})}} = sign(y_i - F(\vec{x_i}))$$

#### Huber loss


$$
L(y, F) =\begin{cases}\frac{(y-F)^2}{2} \qquad \left|y-F\right| \leq \delta, \\ \delta(\left|y-F\right| - \frac{\delta}{2} \qquad
\left|y-F\right| \gt \delta \end{cases} \\
-g(\vec{x_i}) = -\frac{\partial J}{\partial{F(\vec{x_i})}} = \begin{cases} y_i-F(\vec{x_i}) \qquad \left|y_i-F(\vec{x_i})\right| \leq \delta, \\ \delta \cdot sign(\left|y-F\right| \qquad
\left|y_i-F(\vec{x_i})\right| \gt \delta \end{cases}$$

### Boosting for Classification

An simple example will be the best way to convey the method of boosted decision trees for classification

**Example**

We will try to classify a dataset into one of three categories $\{A, B, C\}$

In [2]:
import pandas as pd
sample_data = {"x": ["x%d" % i for i in xrange(5)], "y": ["A", "C", "B", "B", "A"]}
df = pd.DataFrame(sample_data)
df.head()

,x,y
0,x0,A
1,x1,C
2,x2,B
3,x3,B
4,x4,A


Turn the output data into a probability distribution matrix

In [5]:
Y = pd.get_dummies(df.y)
Y

,A,B,C
0,1,0,0
1,0,0,1
2,0,1,0
3,0,1,0
4,1,0,0


For each category, we will train a boosted decision tree to predict a score for each data point. A higher score will lead to higher likelihood of predicting that category. 